# 🤖 Fine-tuning Nous Hermes 2 (Mistral 7B) com Unsloth + GGUF

In [ ]:
!pip uninstall -y numpy torch torchvision torchaudio unsloth
!pip install numpy==1.24.4
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install unsloth[cu118] transformers accelerate peft bitsandbytes trl sentencepiece scipy

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "NousResearch/Nous-Hermes-2-Mistral-7B-DPO",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="fine_tuning_dataset_brasil_with_output.jsonl")

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    dataset_text_field = "input",
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,
        logging_steps = 10,
        save_strategy = "no",
        learning_rate = 2e-4,
        bf16 = False,
        fp16 = True,
        output_dir = "./treinamento_assistente",
    ),
)

trainer.train()

In [ ]:
FastLanguageModel.for_inference(model)
model.save_pretrained("modelo_final_merged")
tokenizer.save_pretrained("modelo_final_merged")

In [ ]:
!pip install git+https://github.com/ggerganov/transformers.git
!transformers-to-gguf -m modelo_final_merged -o modelo_assistente_brasil.gguf